To Do:
- create downloadable csv for each jurisdiction
- deal with duble field entries esp from the simple normalizing

In [2]:
jurs = [
        "CA",
        "MI",
        "UT",
        "ME"
        ]

In [3]:
import pandas as pd
import urllib.request, json 
from urllib.request import urlopen
import re
from bs4 import BeautifulSoup
import PyPDF2
from PyPDF2 import PdfFileWriter

import pikepdf

import os
from os import walk
import os.path
from os import path
import numpy as np
#!pip install py-readability-metrics
from readability import Readability
#!python -m nltk.downloader punkt

import time
from datetime import date

import spacy
from sklearn.preprocessing import normalize

from joblib import dump, load


import networkx as nx

import math

from sklearn.metrics.pairwise import cosine_similarity# define matrix with all zero values

#import nltk
#nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [4]:
nlp = spacy.load('en_core_web_lg')

In [5]:
def vectorize(text):
    output = nlp(str(text)).vector   
    return output

def norm(row):
    try:
        matrix = row.reshape(1,-1).astype(np.float64)
        return normalize(matrix, axis=1, norm='l1')[0]
    except Exception as e: 
        print("===================")
        print(row)
        print("-------------------")
        print(e)
        print("===================")
        #return np.NaN

In [6]:
def regex_norm_field(text):
    # Takes an auto-generated form field name and uses
    # regex to convert it into an Assembly Line standard field.
    # See https://suffolklitlab.org/docassemble-AssemblyLine-documentation/docs/label_variables/

    regex_list = [

        # Personal info
        ## Name & Bio
        ["^((My|Full( legal)?) )?Name$","users1_name"],
        ["^(Typed or )?Printed Name\s?\d*$","users1_name"],
        ["^(DOB|Date of Birth|Birthday)$","users1_birthdate"],
        ## Address
        ["^(Street )?Address$","users1_address_line_one"],
        ["^City State Zip$","users1_address_line_two"],
        ["^City$","users1_address_city"],
        ["^State$","users1_address_state"],
        ["^Zip( Code)?$","users1_address_zip"],
        ## Contact
        ["^(Phone|Telephone)$","users1_phone_number"],
        ["^Email( Adress)$","users1_email"],
        
        # Parties
        ["plaintiff","plantiff1_name"],
        ["defendant","defendant1_name"],
        ["petitioners","petitioners1_name"],
        ["respondents","respondents1_name"],

        # Court info
        ["^(Court\s)?Case\s?(No|Number)?\s?A?$","docket_number"],
        ["^File\s?(No|Number)?\s?A?$","docket_number"],

        # Form info
        ["^(Signature|Sign( here)?)\s?\d*$","users1_signature"],
        ["^Date\s?\d*$","signature_date"],
    ]

    for regex in regex_list:
        text = re.sub(regex[0],regex[1],text, flags=re.IGNORECASE)
    return text

included_fields = load('../data/processed/ML/norm_fields/included_fields.joblib') 
jurisdictions = load('../data/processed/ML/norm_fields/jurisdictions.joblib') 
groups = load('../data/processed/ML/norm_fields/groups.joblib') 
clf_field_names = load('../data/processed/ML/norm_fields/clf_field_names.joblib') 

stop_words = set(stopwords.words('english'))

def reformat_field(text,max_length=30):
    # h/t https://towardsdatascience.com/nlp-building-a-summariser-68e0c19e3a93
    
    #print(text)
    
    orig_title = text.lower()
    orig_title = re.sub("[^a-zA-Z]+"," ",orig_title)
    orig_title_words = orig_title.split()
   
    deduped_sentence = []
    for word in orig_title_words:
        if word not in deduped_sentence:
            deduped_sentence.append(word)
            
    filtered_sentence = [w for w in deduped_sentence if not w.lower() in stop_words]   

    filtered_title_words = filtered_sentence
    
    characters = len(' '.join(filtered_title_words))
    
    if characters > 0:

        words = len(filtered_title_words)
        av_word_len = math.ceil(len(' '.join(filtered_title_words))/len(filtered_title_words))
        x_words = math.floor((max_length)/av_word_len)


        sim_mat = np.zeros([len(filtered_title_words),len(filtered_title_words)])# will populate it with cosine_similarity values 
        # for each word compared to other
        for i in range(len(filtered_title_words)):
            for j in range(len(filtered_title_words)):
                if i != j:
                    sim_mat[i][j] = cosine_similarity(nlp(filtered_title_words[i]).vector.reshape(1,300), nlp(filtered_title_words[j]).vector.reshape(1,300))[0,0]

        try:
            nx_graph = nx.from_numpy_array(sim_mat)
            scores = nx.pagerank(nx_graph)# print final values of words
            sorted_scores = sorted(scores.items(), key=lambda item: item[1], reverse=True)

            if x_words > len(scores):
                x_words=len(scores)

            i = 0
            new_title = ""
            for x in filtered_title_words:
                #print(scores[i],sorted_scores[x_words][1])
                if scores[i] >= sorted_scores[x_words-1][1]: 
                    if len(new_title)>0: new_title+="_"
                    new_title += x
                i+=1

            return new_title
        except:
            return '_'.join(filtered_title_words)
    else:
        if re.search("^(\d+)$", text):
            return "unknown"
        else:
            return re.sub("\s+","_",text.lower())
    

def normalize_name(jur,group,n,per,last_field,this_field):
    
    # Add hard coded conversions maybe by calling a function
    # if returns 0 then fail over to ML or otherway around poor prob -> check hard-coded

    if this_field not in included_fields:
        this_field = reCase(this_field)

        out_put = regex_norm_field(this_field)
        conf = 1.0

        if out_put==this_field:
            params = []
            for item in jurisdictions:
                if jur== item:
                    params.append(1)
                else:
                    params.append(0)
            for item in groups:
                if group== item:
                    params.append(1)
                else:
                    params.append(0)
            params.append(n)
            params.append(per)
            for vec in norm(vectorize(this_field)):
                params.append(vec)
            #for vec in norm(vectorize(last_field)):
            #    params.append(vec)

            for item in included_fields:
                if last_field==item:
                    params.append(1)
                else:
                    params.append(0)

            pred = clf_field_names.predict([params])
            prob = clf_field_names.predict_proba([params])

            conf = prob[0].tolist()[prob[0].tolist().index(max(prob[0].tolist()))]
            out_put = pred[0]
            
    else:
        out_put = this_field
        conf = 1
            
    if out_put in included_fields:
        if conf >= 0:
            return "*"+out_put,conf #+"| was <i>%s</i> (%.2f conf)"%(this_field,conf) #, conf
        else:
            return reformat_field(this_field),conf #+"| was <i>%s</i> (%.2f conf)"%(this_field,conf) #, conf
    else:
        return reformat_field(this_field),conf #+"| was <i>%s</i> (%.2f conf)"%(this_field,conf) #, conf


In [7]:
def reCase(text):
    output = re.sub("(\w|\d)(_|-)(\w|\d)","\\1 \\3",text.strip())
    output = re.sub("([a-z])([A-Z]|\d)","\\1 \\2",output)
    output = re.sub("(\d)([A-Z]|[a-z])","\\1 \\2",output)
    return output

In [8]:
def read_pdf (file):
    try:
        pdfFile = PyPDF2.PdfFileReader(open(file, "rb"))
        if pdfFile.isEncrypted:
            try:
                pdfFile.decrypt('')
                #print ('File Decrypted (PyPDF2)')
            except:
                #
                #
                # This didn't go so well on my Windows box so I just ran this in the pdf folder's cmd:
                # for %f in (*.*) do copy %f temp.pdf /Y && "C:\Program Files (x86)\qpdf-8.0.2\bin\qpdf.exe" --password="" --decrypt temp.pdf %f
                #
                #
                #
                
                command="cp "+file+" tmp/temp.pdf; qpdf --password='' --decrypt tmp/temp.pdf "+file
                os.system(command)
                #print ('File Decrypted (qpdf)')
                #re-open the decrypted file
                pdfFile = PyPDF2.PdfFileReader(open(file, "rb"))
        text = ""
        for page in pdfFile.pages:
            text = text + " " + page.extractText()
        return text
    except:
        return ""


In [107]:
def parse_form(fileloc,title=None,jur=None,cat=None,normalize=1):
    f = PyPDF2.PdfFileReader(fileloc)
    
    if f.isEncrypted:
        pdf = pikepdf.open(fileloc, allow_overwriting_input=True)
        pdf.save(fileloc)
        f = PyPDF2.PdfFileReader(fileloc)
    
    npages = f.getNumPages()
    ff = f.getFields()
    if ff:
        fields = list(ff.keys())
    else:
        fields = []
    f_per_page = len(fields)/npages
    text = read_pdf(fileloc)
    try:
        readbility = int(Readability(text).flesch_kincaid().grade_level)
    except:
        readbility = None
    
    if title is None:
        title = reCase(re.search("(.*)\n",text).group(1).strip())
        
    if normalize==1:
        i = 0 
        length = len(fields)
        last = "null"
        new_fields = []
        new_fields_conf = []
        for field in fields:
            #print(jur,cat,i,i/length,last,field)
            this_field,this_conf = normalize_name(jur,cat,i,i/length,last,field)
            new_fields.append(this_field)
            new_fields_conf.append(this_conf)
            last = field
        
        new_fields = [v + "__" + str(new_fields[:i].count(v) + 1) if new_fields.count(v) > 1 else v for i, v in enumerate(new_fields)]
    else:
        new_fields = fields
    
    stats = {
            "title":title,
            "category":cat,
            "pages":npages,
            "reading grade level": readbility,
            "avg fields per page": f_per_page,
            "fields":new_fields,
            "fields_conf":new_fields_conf,
            "fields_old":fields
            }    
    
    try:
        if 1==1:
            my_pdf = pikepdf.Pdf.open(fileloc)
            fields_too = my_pdf.Root.AcroForm.Fields #[0]["/Kids"][0]["/Kids"][0]["/Kids"][0]["/Kids"]

            #print(repr(fields_too))
            k =0
            for field in new_fields:
                #print(k,field)
                fields_too[k].T = re.sub("^\*","",field)
                k+=1

            #f2.T = 'new_hospital_name'
            filename = re.search("\/(\w*\.pdf)$",fileloc).groups()[0]
            my_pdf.save('../data/processed/forms/%s'%(filename))
            #my_pdf.save('%s'%(filename))
        else:
            file = PdfFileWriter()

            first_page = f.getPage(0)

            file.cloneDocumentFromReader(f)
            #file.appendPagesFromReader(f)

            x ={}
            for y in ff:
                x[y]=""

            #print(x)

            file.updatePageFormFieldValues(first_page,x)

            output = open('blankPdf.pdf', 'wb')
            file.write(output)  
    except:
        error = "could not change form fields"
    
    return stats

In [103]:
#parse_form("../data/processed/www.utcourts.gov/forms/898269a99ff1c65be10b1ae35bb34ba469fc14b7301b7ed7b126d195.pdf",title=None,jur="UT",cat=None,normalize=1)
#parse_form("../data/processed/www.utcourts.gov/forms/2532cd2b6d3aaff8c47726a0abd168fb4e5cdb4977c065cd27bde8c7.pdf",title=None,jur="UT",cat=None,normalize=1)
#parse_form("../data/processed/www.utcourts.gov/forms/6ec7576210513907e699b5adf3397639507c688801a60bc34c201984.pdf",title=None,jur="UT",cat=None,normalize=1)
parse_form("../data/processed/mjbportal.courts.maine.gov/forms/1519fe450d870a36a428a0b006c0665a.pdf",title=None,jur="UT",cat=None,normalize=1)
#parse_form("../data/processed/www.courts.ca.gov/forms/3979f1c1c9f165ccac026b26cf20252c.pdf",title=None,jur="UT",cat=None,normalize=1)

{'title': 'PA 005, Rev. 11/08',
 'category': None,
 'pages': 1,
 'reading grade level': 12,
 'avg fields per page': 48.0,
 'fields': ['name__1',
  'home_address',
  'apartment_floor',
  'color_house_description',
  'living_another_person_persons',
  'telephone_homeworkcell',
  'hours_defendant_likely_home',
  'name_employer',
  'work_address',
  'work_schedule_x_days',
  'am',
  'am_2',
  'pm__1',
  's',
  'm',
  't',
  'w',
  'th',
  'f',
  's_2',
  'hours_worked',
  'pm__2',
  'birth_date',
  'make_year',
  'height',
  'type_model',
  'weight',
  'color',
  'hair_color',
  'eye_color',
  'registration_state__1',
  'registration_state__2',
  'gender',
  'white',
  'black',
  'asian_pacific_islander',
  'amer_indian_alaskan_native',
  'unknown',
  'anyone_help_locate__1',
  'anyone_help_locate__2',
  'defendant_firearm_weapon',
  'weapon_usually_kept',
  'defendant_history_violence',
  'anything_else_know__1',
  'anything_else_know__2',
  'name__2',
  'address_unless_confidential',
  '

In [104]:
parse_form("../data/processed/www.courts.ca.gov/forms/3979f1c1c9f165ccac026b26cf20252c.pdf",title=None,jur="UT",cat=None,normalize=1)

could not change form fields


{'title': 'Judicial Council of California,   Rev. September 1, 2021, Mandatory Form   Family Code, §§ 170Œ180, 7660Œ7671, 7822, 7892.5, 7960, 8601.5,',
 'category': None,
 'pages': 6,
 'reading grade level': 17,
 'avg fields per page': 38.333333333333336,
 'fields': ['button__1',
  'p_footer',
  'page_area',
  'print',
  'save',
  'reset',
  'warning',
  'footer_buttons',
  'page__1',
  'page_set',
  'court_info_ft',
  'case_number_ft',
  'stamp_court_case',
  'p_header',
  'parent',
  'parent_b',
  'parent_rel',
  'parent_street',
  'parent_city',
  'parent_state',
  'parent_zip',
  'parent_tel',
  'item_tf',
  'li__1',
  'list__1',
  'cty_adopt_parents_cb',
  'cty_child_cb',
  'cty_placement_office_cb',
  'cty_investigating_office_cb',
  'plc_birth_parents_cb',
  'cty_birth_parents_pet_cb',
  'cty_child_freed_cb',
  'list__2',
  'hearing_date',
  'hearing_time',
  'hearing_dept',
  'hearing_rm',
  'name_add_tf',
  'hearing_info',
  'hearing_sub',
  'type_agency_cb',
  'type_agency_na

In [86]:
stats[0]["/Kids"][0]["/Kids"][0]["/Kids"][0]["/Kids"][0].T

pikepdf.String("Button1[0]")

In [91]:
stats[0]["/Kids"][0]["/Kids"][0]["/Kids"][0].T

pikepdf.String("P1Footer[0]")

In [96]:
stats[0]["/Kids"][0]["/Kids"][1]["/Kids"][0].T

pikepdf.String("FooterButtons[0]")

In [94]:
stats[0]["/Kids"][0]["/Kids"][1].T

pikepdf.String("Page3[0]")

In [68]:
my_pdf = pikepdf.Pdf.open("../data/processed/www.courts.ca.gov/forms/3979f1c1c9f165ccac026b26cf20252c.pdf")
fields_too = my_pdf.Root.AcroForm.Fields#[0]["/Kids"][0]["/Kids"][0]["/Kids"][0]["/Kids"]

In [77]:
fields_too[0]["/Kids"][0]["/Kids"][0]["/Kids"][0]["/Kids"][0].T

pikepdf.String("Button1[0]")

In [108]:
#files_df_ut = pd.read_csv("../data/raw/www.utcourts.gov/form_data.csv")
#files_df_mi = pd.read_csv("../data/raw/www.courts.michigan.gov/form_data.csv")
#files_df_me = pd.read_csv("../data/raw/mjbportal.courts.maine.gov/form_data.csv")
#files_df_ca = pd.read_csv("../data/raw/www.courts.ca.gov/form_data.csv")

#df = pd.concat([files_df_ut,files_df_mi,files_df_me],ignore_index=True)
df = pd.read_csv("../data/raw/www.courts.ca.gov/form_data.csv")
df.head()

,id,jurisdiction,source,title,group,url,filename,downloaded
0,a895e7c54d5a41a458aef57e16da5f1d,CA,www.courts.ca.gov,How to Adopt a Child in California,ADOPT,https://www.courts.ca.gov/documents/adopt050in...,adopt050info.pdf,2021-11-17
1,3979f1c1c9f165ccac026b26cf20252c,CA,www.courts.ca.gov,Adoption Request,ADOPT,https://www.courts.ca.gov/documents/adopt200.pdf,adopt200.pdf,2021-11-17
2,9f441bfaca115775970cf03550e0c78e,CA,www.courts.ca.gov,Declaration Confirming Parentage in Stepparent...,ADOPT,https://www.courts.ca.gov/documents/adopt205.pdf,adopt205.pdf,2021-11-17
3,72e33ceb8675e0b13a39d488ffd2de48,CA,www.courts.ca.gov,Declaration Confirming Parentage in Stepparent...,ADOPT,https://www.courts.ca.gov/documents/adopt206.pdf,adopt206.pdf,2021-11-17
4,f020836e1802974aa0f6b2d6b8be5bb5,CA,www.courts.ca.gov,Adoption Agreement,ADOPT,https://www.courts.ca.gov/documents/adopt210.pdf,adopt210.pdf,2021-11-17


In [ ]:
df["pages"] = ""
df["fields"] = ""
df["fields_conf"] = ""
df["fields_old"] = ""
df["f_per_p"] = ""
df["reading"] = ""

for index,row in df.iterrows():
    if (row["pages"] == "") & (row["jurisdiction"] in jurs):
        try:
            stats = parse_form("../data/processed/"+row["source"]+"/forms/"+row["id"]+".pdf",row["title"],row["jurisdiction"],row["group"],1)
            df.at[index, 'pages'] = stats["pages"]
            df.at[index, 'fields'] = stats["fields"]
            df.at[index, 'fields_conf'] = stats["fields_conf"]
            df.at[index, 'fields_old'] = stats["fields_old"]
            df.at[index, 'f_per_p'] = stats["avg fields per page"]
            df.at[index, 'reading'] = stats["reading grade level"]
            #print(index)
        except:
            print("error: "+"../data/raw/"+row["source"]+"/forms/"+row["id"]+".pdf")
        
df

In [ ]:
df1 = pd.read_csv("../data/processed/form_data.csv")
df1

In [ ]:
df2 = df[(df["jurisdiction"]=="CA") & (df["pages"]!="")]
df2

In [ ]:
dfall = pd.concat([df1,df2],ignore_index=True)
dfall

In [28]:
#dfall.to_csv("../data/processed/form_data.csv", index=False, encoding="utf-8")    
df.to_csv("../data/processed/form_data.csv", index=False, encoding="utf-8")    

In [ ]:
#os.system('cp ../data/processed/www.utcourts.gov/forms/* ../data/processed/forms/')
#os.system('cp ../data/processed/www.courts.michigan.gov/forms/* ../data/processed/forms/')